# Analyze Metrics and Conduct Inference.ipynb

In [19]:
import pandas as pd
from statsmodels.stats.contingency_tables import mcnemar 
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats


In [20]:
# Load Combined_Data.xlsx
combined_data = pd.read_excel('Combined_Data.xlsx')

# GSM8k data - limit to task = "gsm8k"
gsm8k_data = combined_data[combined_data['task'] == 'gsm8k']

# Creative writing data - limit to task = "cw"
cw_data = combined_data[combined_data['task'] == 'cw']


## Accuracy/Quality

In [21]:
# Average accuracy_quality by model, method, task
# Also get variance
avg_accuracy_quality_with_variance = combined_data[['model', 'method', 'task', 'accuracy_quality']].groupby(['model', 'method', 'task']).agg(['mean', 'var'])['accuracy_quality'].reset_index()

# Combine mean and variance into one column that is a string with the mean and then the variance in parentheses
avg_accuracy_quality_with_variance['accuracy_quality'] = avg_accuracy_quality_with_variance.apply(lambda row: str(round(row['mean'], 2)) + ' (' + str(round(row['var'], 2)) + ')', axis=1)

avg_accuracy_quality_with_variance


,model,method,task,mean,var,accuracy_quality
0,gpt4,ape_zero_shot_cot,cw,5.46,6.210505,5.46 (6.21)
1,gpt4,ape_zero_shot_cot,gsm8k,0.93,0.065758,0.93 (0.07)
2,gpt4,direct_prompting,cw,5.71,7.298889,5.71 (7.3)
3,gpt4,direct_prompting,gsm8k,0.73,0.199091,0.73 (0.2)
4,gpt4,least_to_most,cw,5.54,5.907475,5.54 (5.91)
5,gpt4,least_to_most,gsm8k,0.95,0.047980,0.95 (0.05)
6,gpt4,manual_cot,cw,6.33,5.193030,6.33 (5.19)
7,gpt4,manual_cot,gsm8k,0.93,0.065758,0.93 (0.07)
8,gpt4,manual_few_shot,cw,6.11,6.220101,6.11 (6.22)
9,gpt4,manual_few_shot,gsm8k,0.49,0.252424,0.49 (0.25)


In [22]:
# Loop over cases where task = "cw" and output histograms of accuracy quality by model and method
# Loop over models
for model in cw_data['model'].unique():
    # Loop over methods
    for method in cw_data['method'].unique():
        # Get data for this model and method
        data = cw_data[(cw_data['model'] == model) & (cw_data['method'] == method)]
        # Plot histogram
        plt.hist(data['coherence_1_incoherent_10_very_coherent'], bins = 10)
        # Ticks for values of 1, 2, ... 10
        plt.xticks(range(1, 11))
        plt.title(model + ' - ' + method)
        plt.xlabel('Accuracy Quality')
        plt.ylabel('Count')
        # Save histogram
        plt.savefig('../Output/cw_acc_qual_' + model + '_' + method + '.png')
        plt.close()


In [23]:
# Pivot table - column method should go wide
avg_accuracy_quality_pivot = avg_accuracy_quality_with_variance.drop(columns=['mean', 'var']).set_index(['model', 'task', 'method']).unstack()
#pivot_table(index=['model', 'task'], columns='method', values='accuracy_quality').reset_index()

# Fix axis
avg_accuracy_quality_pivot = avg_accuracy_quality_pivot.rename_axis([None, None], axis=1).reset_index()

# Sort rows by task - gsm8k task first, then cw
# Sort by model - text-davinci-003 first, then gpt4
avg_accuracy_quality_pivot = avg_accuracy_quality_pivot.sort_values(by=['task', 'model'], ascending=[True, True]).reset_index().drop(columns=['index'])

print(avg_accuracy_quality_pivot.columns)

flattened_cols = [''.join(col).strip().replace('accuracy_quality', '') for col in avg_accuracy_quality_pivot.columns.values]
avg_accuracy_quality_pivot.columns = flattened_cols

print(avg_accuracy_quality_pivot)

# Order columns: direct_prompting, zero_shot_cot, ape_zero_shot_cot, tree_of_thought, self_refine, least_to_most, manual_few_shot, manual_cot
avg_accuracy_quality_pivot = avg_accuracy_quality_pivot[['model', 'task', 'direct_prompting', 'zero_shot_cot', 'ape_zero_shot_cot', 'tree_of_thought', 'self_refine', 'least_to_most', 'manual_few_shot', 'manual_cot']]

# Output to LaTeX
avg_accuracy_quality_pivot.to_latex('../Output/avg_accuracy_quality_pivot.tex', index=False)


MultiIndex([(           'model',                  ''),
            (            'task',                  ''),
            ('accuracy_quality', 'ape_zero_shot_cot'),
            ('accuracy_quality',  'direct_prompting'),
            ('accuracy_quality',     'least_to_most'),
            ('accuracy_quality',        'manual_cot'),
            ('accuracy_quality',   'manual_few_shot'),
            ('accuracy_quality',       'self_refine'),
            ('accuracy_quality',   'tree_of_thought'),
            ('accuracy_quality',     'zero_shot_cot')],
           )
  model   task ape_zero_shot_cot direct_prompting least_to_most   manual_cot  \
0  gpt4     cw       5.46 (6.21)       5.71 (7.3)   5.54 (5.91)  6.33 (5.19)   
1   td3     cw        3.9 (5.36)      4.46 (7.79)   4.45 (5.28)  4.14 (4.89)   
2  gpt4  gsm8k       0.93 (0.07)       0.73 (0.2)   0.95 (0.05)  0.93 (0.07)   
3   td3  gsm8k       0.49 (0.25)      0.23 (0.18)   0.67 (0.95)   0.6 (0.24)   

  manual_few_shot  self_refine tree

C:\Users\ijyli\AppData\Local\Temp\ipykernel_30492\280831353.py:10: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  avg_accuracy_quality_pivot = avg_accuracy_quality_pivot.sort_values(by=['task', 'model'], ascending=[True, True]).reset_index().drop(columns=['index'])


In [24]:
# Perform McNemar's Test on GSM8k data

# Get in terms of number of questions answered correctly: sum accuracy_quality column by model, method, task
gsm8k_num_correct= gsm8k_data[['model', 'method', 'task', 'correct']].groupby(['model', 'method', 'task']).sum()['correct'].reset_index()
# Rename column to num_correct
gsm8k_num_correct = gsm8k_num_correct.rename(columns={'correct': 'num_correct'})

# Make another table with counts of questions missed (number of observations minus accuracy_quality sum) by model, method, task
gsm8k_num_obs = gsm8k_data[['model', 'method', 'task', 'correct']].groupby(['model', 'method', 'task']).count()['correct'].reset_index()
# Rename column to num_obs
gsm8k_num_obs = gsm8k_num_obs.rename(columns={'correct': 'num_obs'})
# Join together by model, method, task
gsm8k_correct_incorrect_obs = gsm8k_num_correct.merge(gsm8k_num_obs, on=['model', 'method', 'task'], how='left')
# Table should have model, method, task, num_correct, num_incorrect, num_obs
gsm8k_correct_incorrect_obs['num_incorrect'] = gsm8k_correct_incorrect_obs['num_obs'] - gsm8k_correct_incorrect_obs['num_correct']

gsm8k_correct_incorrect_obs


,model,method,task,num_correct,num_obs,num_incorrect
0,gpt4,ape_zero_shot_cot,gsm8k,93.0,100,7.0
1,gpt4,direct_prompting,gsm8k,73.0,100,27.0
2,gpt4,least_to_most,gsm8k,95.0,100,5.0
3,gpt4,manual_cot,gsm8k,93.0,100,7.0
4,gpt4,manual_few_shot,gsm8k,49.0,100,51.0
5,gpt4,self_refine,gsm8k,89.0,100,11.0
6,gpt4,tree_of_thought,gsm8k,40.0,100,60.0
7,gpt4,zero_shot_cot,gsm8k,95.0,100,5.0
8,td3,ape_zero_shot_cot,gsm8k,49.0,100,51.0
9,td3,direct_prompting,gsm8k,23.0,100,77.0


### Testing Accuracy

In [25]:
# Function to perform McNemar's test
# Accepts argument of name of model, method, task

def perform_mcnemar(model, method, task):
    
    # Get direct prompting data
    dp_data = gsm8k_data[(combined_data['model'] == model) & (combined_data['method'] == 'direct_prompting') & (combined_data['task'] == task)][['conversation_number', 'correct']]
    # Rename correct to dp_correct
    dp_data = dp_data.rename(columns={'correct': 'dp_correct'})

    # Get method data
    method_data = gsm8k_data[(combined_data['model'] == model) & (combined_data['method'] == method) & (combined_data['task'] == task)][['conversation_number', 'correct']]
    # Rename correct to method_correct
    method_data = method_data.rename(columns={'correct': 'method_correct'})

    # Join together by conversation_number
    joined_data = dp_data.merge(method_data, on='conversation_number', how='left')

    # Fill contingency table cells
    dp_correct_method_correct = joined_data[(joined_data['dp_correct'] == True) & (joined_data['method_correct'] == True)].shape[0]
    dp_correct_method_incorrect = joined_data[(joined_data['dp_correct'] == True) & (joined_data['method_correct'] == False)].shape[0]
    dp_incorrect_method_correct = joined_data[(joined_data['dp_correct'] == False) & (joined_data['method_correct'] == True)].shape[0]
    dp_incorrect_method_incorrect = joined_data[(joined_data['dp_correct'] == False) & (joined_data['method_correct'] == False)].shape[0]
    
    # Create a dataset
    # Rows for dp correct and incorrect
    # Columns for method correct and incorrect
    data = [[dp_correct_method_correct, dp_correct_method_incorrect], 
            [dp_incorrect_method_correct, dp_incorrect_method_incorrect]]
  
    print('McNemar\'s Test (Exact) for ' + model + ' ' + method + ' ' + task)
    print(data)

    # McNemar's Test, exact, without any continuity correction 
    print('No continuity correction')
    print(mcnemar(data, exact=True, correction=False)) 
    ncc_result = mcnemar(data, exact=True, correction=False)
    ncc_statistic = ncc_result.statistic
    ncc_p_value = ncc_result.pvalue

    # McNemar's Test with the continuity correction 
    print('With continuity correction')
    print(mcnemar(data, exact=True, correction=True)) 
    cc_result = mcnemar(data, exact=True, correction=True)
    cc_statistic = cc_result.statistic
    cc_p_value = cc_result.pvalue

    # Return data
    return ncc_statistic, ncc_p_value, cc_statistic, cc_p_value


In [26]:
# Add McNemar's Test results to table
# Iterate over all combinations of model, method, task
# Except for method = "direct_prompting" and task = "cw"
# For each combination, perform McNemar's Test and add to table

# Create combos of model, method, task - all unique combinations of these three in combined_data
# Get unique values of model, method, task
models = combined_data['model'].unique()
methods = combined_data['method'].unique()
tasks = combined_data['task'].unique()
# Create list of all combinations of model, method, task
combinations = [(model, method, task) for model in models for method in methods for task in tasks]

# Add as rows to a dataframe
# Create empty dataframe
mcnemars_results = pd.DataFrame(columns=['model', 'method', 'task', 'statistic', 'pvalue', 'statistic_with_correction', 'pvalue_with_correction'])

# Iterate over combinations
for model, method, task in combinations:
    # Skip if method = "direct_prompting" or task = "cw"
    if method == 'direct_prompting' or task == 'cw':
        continue
    # Perform McNemar's Test
    statistic_without_correction, pvalue_without_correction, statistic_with_correction, pvalue_with_correction = perform_mcnemar(model, method, task)
    # Add to table
    mcnemars_results = pd.concat([mcnemars_results, pd.DataFrame([[model, method, task, statistic_without_correction, pvalue_without_correction, statistic_with_correction, pvalue_with_correction]], columns=['model', 'method', 'task', 'statistic', 'pvalue', 'statistic_with_correction', 'pvalue_with_correction'])])

mcnemars_results


McNemar's Test (Exact) for td3 zero_shot_cot gsm8k
[[13, 10], [49, 28]]
No continuity correction
pvalue      2.706279703008907e-07
statistic   10.0
With continuity correction
pvalue      2.706279703008907e-07
statistic   10.0
McNemar's Test (Exact) for td3 ape_zero_shot_cot gsm8k
[[11, 12], [38, 39]]
No continuity correction
pvalue      0.00030586400160359517
statistic   12.0
With continuity correction
pvalue      0.00030586400160359517
statistic   12.0
McNemar's Test (Exact) for td3 least_to_most gsm8k
[[15, 7], [43, 34]]
No continuity correction
pvalue      2.098677427397888e-07
statistic   7.0
With continuity correction
pvalue      2.098677427397888e-07
statistic   7.0
McNemar's Test (Exact) for td3 manual_few_shot gsm8k
[[11, 12], [7, 70]]
No continuity correction
pvalue      0.359283447265625
statistic   7.0
With continuity correction
pvalue      0.359283447265625
statistic   7.0
McNemar's Test (Exact) for td3 manual_cot gsm8k
[[16, 7], [44, 33]]
No continuity correction
pvalue   

C:\Users\ijyli\AppData\Local\Temp\ipykernel_30492\1577853410.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dp_data = gsm8k_data[(combined_data['model'] == model) & (combined_data['method'] == 'direct_prompting') & (combined_data['task'] == task)][['conversation_number', 'correct']]
C:\Users\ijyli\AppData\Local\Temp\ipykernel_30492\1577853410.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  method_data = gsm8k_data[(combined_data['model'] == model) & (combined_data['method'] == method) & (combined_data['task'] == task)][['conversation_number', 'correct']]
C:\Users\ijyli\AppData\Local\Temp\ipykernel_30492\1577853410.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dp_data = gsm8k_data[(combined_data['model'] == model) & (combined_data['method'] == 'direct_prompting') & (combined_data['task'] == task)][['conversation_number', 'correct']]
C:\Users\ijyli\AppData\Local\Temp\ipykernel_

,model,method,task,statistic,pvalue,statistic_with_correction,pvalue_with_correction
0,td3,zero_shot_cot,gsm8k,10.0,2.706280e-07,10.0,2.706280e-07
0,td3,ape_zero_shot_cot,gsm8k,12.0,3.058640e-04,12.0,3.058640e-04
0,td3,least_to_most,gsm8k,7.0,2.098677e-07,7.0,2.098677e-07
0,td3,manual_few_shot,gsm8k,7.0,3.592834e-01,7.0,3.592834e-01
0,td3,manual_cot,gsm8k,7.0,1.211526e-07,7.0,1.211526e-07
0,td3,tree_of_thought,gsm8k,13.0,1.000000e+00,13.0,1.000000e+00
0,td3,self_refine,gsm8k,4.0,5.488281e-01,4.0,5.488281e-01
0,gpt4,zero_shot_cot,gsm8k,1.0,2.980232e-06,1.0,2.980232e-06
0,gpt4,ape_zero_shot_cot,gsm8k,1.0,1.096725e-05,1.0,1.096725e-05
0,gpt4,least_to_most,gsm8k,0.0,4.768372e-07,0.0,4.768372e-07


In [27]:
# Rows that are significant at p < 0.05
mcnemars_results[mcnemars_results['pvalue'] < 0.05]


,model,method,task,statistic,pvalue,statistic_with_correction,pvalue_with_correction
0,td3,zero_shot_cot,gsm8k,10.0,2.706280e-07,10.0,2.706280e-07
0,td3,ape_zero_shot_cot,gsm8k,12.0,3.058640e-04,12.0,3.058640e-04
0,td3,least_to_most,gsm8k,7.0,2.098677e-07,7.0,2.098677e-07
0,td3,manual_cot,gsm8k,7.0,1.211526e-07,7.0,1.211526e-07
0,gpt4,zero_shot_cot,gsm8k,1.0,2.980232e-06,1.0,2.980232e-06
0,gpt4,ape_zero_shot_cot,gsm8k,1.0,1.096725e-05,1.0,1.096725e-05
0,gpt4,least_to_most,gsm8k,0.0,4.768372e-07,0.0,4.768372e-07
0,gpt4,manual_few_shot,gsm8k,9.0,2.715392e-04,9.0,2.715392e-04
0,gpt4,manual_cot,gsm8k,1.0,1.096725e-05,1.0,1.096725e-05
0,gpt4,tree_of_thought,gsm8k,8.0,1.964654e-06,8.0,1.964654e-06


In [28]:
# Rows that are not significant at p < 0.05
mcnemars_results[mcnemars_results['pvalue'] >= 0.05]


,model,method,task,statistic,pvalue,statistic_with_correction,pvalue_with_correction
0,td3,manual_few_shot,gsm8k,7.0,0.359283,7.0,0.359283
0,td3,tree_of_thought,gsm8k,13.0,1.000000,13.0,1.000000
0,td3,self_refine,gsm8k,4.0,0.548828,4.0,0.548828


In [29]:
# Printing gsm8k_correct_incorrect_obs again for reference
gsm8k_correct_incorrect_obs


,model,method,task,num_correct,num_obs,num_incorrect
0,gpt4,ape_zero_shot_cot,gsm8k,93.0,100,7.0
1,gpt4,direct_prompting,gsm8k,73.0,100,27.0
2,gpt4,least_to_most,gsm8k,95.0,100,5.0
3,gpt4,manual_cot,gsm8k,93.0,100,7.0
4,gpt4,manual_few_shot,gsm8k,49.0,100,51.0
5,gpt4,self_refine,gsm8k,89.0,100,11.0
6,gpt4,tree_of_thought,gsm8k,40.0,100,60.0
7,gpt4,zero_shot_cot,gsm8k,95.0,100,5.0
8,td3,ape_zero_shot_cot,gsm8k,49.0,100,51.0
9,td3,direct_prompting,gsm8k,23.0,100,77.0


## Paired t-test for quality, means of other metrics

In [30]:
# Perform paired t-test on creative writing data and all other metrics

# Metrics to t-test: 'creative_writing_score', 'ease_of_evaluation_score', 'conversation_length', 'input_length', 'output_length', 'conversation_cost', 'gsm8k_length_vs_provided', 'length_vs_direct_prompting', 'num_linebreaks', 'num_sentences', 'num_step_i', 'num_1_dot_etc', 'sentence_length', 'fres', 'num_linebreaks_prompts', 'num_sentences_prompts', 'num_step_i_prompts', 'num_1_dot_etc_prompts', 'sentence_length_prompts', 'fres_prompts'
metrics_to_t_test = ['coherence_1_incoherent_10_very_coherent', 'ease_of_review_1_easy_10_hard', 'conversation_length', 'input_length', 'output_length', 'conversation_cost', 'gsm8k_length_vs_provided', 'length_vs_direct_prompting', 'num_linebreaks', 'num_sentences', 'num_step_i', 'num_1_dot_etc', 'sentence_length', 'fres', 'num_linebreaks_prompts', 'num_sentences_prompts', 'num_step_i_prompts', 'num_1_dot_etc_prompts', 'sentence_length_prompts', 'fres_prompts']

# Create table to hold results
# Columns of model, task, method, mean metric, mean dp_metric, statistic, pvalue
t_test_results = pd.DataFrame(columns=['model', 'task', 'method', 'metric', 'dp_mean', 'using_method_mean', 'statistic', 'pvalue'])
# Iterate over model, task, method
for model, task, method in direct_prompting_comparison[['model', 'task', 'method']].drop_duplicates().values:
    # Iterate over metrics_to_t_test
    for metric in metrics_to_t_test:
        # Get data
        # Sort by conversation_number
        data = direct_prompting_comparison[(direct_prompting_comparison['model'] == model) & (direct_prompting_comparison['task'] == task) & (direct_prompting_comparison['method'] == method)].sort_values(by=['conversation_number'])
        # direct_prompting holds the metric when using direct_prompting
        direct_prompting = data['dp_' + metric]
        # using_method holds the metric when using the method
        using_method = data[metric]
        # Perform the paired sample t-test 
        statistic, pvalue = stats.ttest_rel(direct_prompting, using_method)
        # Add to table
        t_test_results = pd.concat([t_test_results, pd.DataFrame([[model, task, method, metric, direct_prompting.mean(), using_method.mean(), statistic, pvalue]], columns=['model', 'task', 'method', 'metric', 'dp_mean', 'using_method_mean', 'statistic', 'pvalue'])])

# Significance if pvalue < 0.05
t_test_results['Significant at 95%'] = t_test_results['pvalue'].apply(lambda x: 'Yes' if x < 0.05 else 'No')

# Sort by metric, model, task, method
t_test_results = t_test_results.sort_values(by=['metric', 'model', 'task', 'method']).reset_index().drop(columns=['index'])

t_test_results


c:\Users\ijyli\anaconda3\envs\anlp\Lib\site-packages\scipy\stats\_axis_nan_policy.py:523: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)


,model,task,method,metric,dp_mean,using_method_mean,statistic,pvalue,Significant at 95%
0,gpt4,cw,ape_zero_shot_cot,coherence_1_incoherent_10_very_coherent,5.71,5.46,0.810999,0.419311,No
1,gpt4,cw,least_to_most,coherence_1_incoherent_10_very_coherent,5.71,5.54,0.607946,0.544615,No
2,gpt4,cw,manual_cot,coherence_1_incoherent_10_very_coherent,5.71,6.33,-2.094400,0.038778,Yes
3,gpt4,cw,manual_few_shot,coherence_1_incoherent_10_very_coherent,5.71,6.11,-1.271348,0.206584,No
4,gpt4,cw,self_refine,coherence_1_incoherent_10_very_coherent,5.71,6.04,-1.062998,0.290369,No
...,...,...,...,...,...,...,...,...,...
555,td3,gsm8k,manual_cot,sentence_length_prompts,NaN,NaN,NaN,NaN,No
556,td3,gsm8k,manual_few_shot,sentence_length_prompts,NaN,NaN,NaN,NaN,No
557,td3,gsm8k,self_refine,sentence_length_prompts,NaN,NaN,NaN,NaN,No
558,td3,gsm8k,tree_of_thought,sentence_length_prompts,NaN,NaN,NaN,NaN,No


In [31]:
# Sort by metric, model, task, Significant at 95%
t_test_results = t_test_results.sort_values(by=['metric', 'model', 'task', 'Significant at 95%']).reset_index().drop(columns=['index'])

t_test_results


,model,task,method,metric,dp_mean,using_method_mean,statistic,pvalue,Significant at 95%
0,gpt4,cw,ape_zero_shot_cot,coherence_1_incoherent_10_very_coherent,5.71,5.46,0.810999,0.419311,No
1,gpt4,cw,least_to_most,coherence_1_incoherent_10_very_coherent,5.71,5.54,0.607946,0.544615,No
2,gpt4,cw,manual_few_shot,coherence_1_incoherent_10_very_coherent,5.71,6.11,-1.271348,0.206584,No
3,gpt4,cw,self_refine,coherence_1_incoherent_10_very_coherent,5.71,6.04,-1.062998,0.290369,No
4,gpt4,cw,manual_cot,coherence_1_incoherent_10_very_coherent,5.71,6.33,-2.094400,0.038778,Yes
...,...,...,...,...,...,...,...,...,...
555,td3,gsm8k,manual_cot,sentence_length_prompts,NaN,NaN,NaN,NaN,No
556,td3,gsm8k,manual_few_shot,sentence_length_prompts,NaN,NaN,NaN,NaN,No
557,td3,gsm8k,self_refine,sentence_length_prompts,NaN,NaN,NaN,NaN,No
558,td3,gsm8k,tree_of_thought,sentence_length_prompts,NaN,NaN,NaN,NaN,No


In [32]:
# Rows that are not significant at p < 0.05
t_test_results[t_test_results['pvalue'] >= 0.05]


,model,task,method,metric,dp_mean,using_method_mean,statistic,pvalue,Significant at 95%
0,gpt4,cw,ape_zero_shot_cot,coherence_1_incoherent_10_very_coherent,5.710000,5.460000,0.810999,0.419311,No
1,gpt4,cw,least_to_most,coherence_1_incoherent_10_very_coherent,5.710000,5.540000,0.607946,0.544615,No
2,gpt4,cw,manual_few_shot,coherence_1_incoherent_10_very_coherent,5.710000,6.110000,-1.271348,0.206584,No
3,gpt4,cw,self_refine,coherence_1_incoherent_10_very_coherent,5.710000,6.040000,-1.062998,0.290369,No
14,td3,cw,least_to_most,coherence_1_incoherent_10_very_coherent,4.460000,4.450000,0.030069,0.976073,No
15,td3,cw,manual_cot,coherence_1_incoherent_10_very_coherent,4.460000,4.140000,0.928556,0.355378,No
16,td3,cw,manual_few_shot,coherence_1_incoherent_10_very_coherent,4.460000,4.970000,-1.630152,0.106247,No
17,td3,cw,self_refine,coherence_1_incoherent_10_very_coherent,4.460000,4.200000,0.877447,0.382368,No
18,td3,cw,zero_shot_cot,coherence_1_incoherent_10_very_coherent,4.460000,3.860000,1.884092,0.062487,No
112,gpt4,cw,self_refine,fres,63.776700,62.369000,1.616096,0.109256,No


In [33]:
# Save t_test_results to Excel
t_test_results.to_excel('t_test_results.xlsx', index=False)


## Adding release date to accuracy quality table (optional version)

In [34]:
# Release date and gains versus direct prompting

# Load Excel file "..\Selection of Prompt Engineering Methods\Hand-Labeled Method and Implementation Considerations.xlsx"
hand_labeled_data = pd.read_excel('..\Selection of Prompt Engineering Methods\Hand-Labeled Method and Implementation Considerations.xlsx')

# Keep columns ss_publication_date, "Prompt Engineering Method"
hand_labeled_data = hand_labeled_data[['ss_publication_date', 'Prompt Engineering Method']]

# Mapping names from this data to technqiue names
# "Few-Shot Learing" -> "manual_few_shot"
# "Chain-of-Thought Prompting" -> "manual_cot"
# "Zero-Shot Chain-of_Thought" -> "zero_shot_cot"
# "Automatic Prompt Engineer" -> "ape_zero_shot_cot"
# "Self-Refine" -> "self_refine"
# "Least-to-Most Prompting" -> "least_to_most"
# "Tree-of-Thought" -> "tree_of_thought"
# Set names in hand_labeled_data to these names
hand_labeled_data['technique_name'] = hand_labeled_data['Prompt Engineering Method'].replace({'Few-Shot Learning': 'manual_few_shot', 'Chain-of-Thought Prompting': 'manual_cot', 'Zero-Shot Chain-of-Thought': 'zero_shot_cot', 'Automatic Prompt Engineer': 'ape_zero_shot_cot', 'Self-Refine': 'self_refine', 'Least-to-Most Prompting': 'least_to_most', 'Tree-of-Thought': 'tree_of_thought'})

# Keep cases where technique_name is not 'Prompt Engineering Method'
hand_labeled_data = hand_labeled_data[hand_labeled_data['technique_name'] != hand_labeled_data['Prompt Engineering Method']]

# Convert ss_publication_date to "YYYY-MM-DD"
hand_labeled_data['ss_publication_date'] = hand_labeled_data['ss_publication_date'].dt.strftime('%Y-%m-%d')

# Drop the item with technique_name = 'tree_of_thought' that has the higher ss_publication_date
# Just keep the row with min ss_publication_date for each technique_name
hand_labeled_data = hand_labeled_data.groupby(['technique_name']).min().reset_index()
print(hand_labeled_data)

# Add ss_publication_date to avg_accuracy_quality_pivot
# The value should be for each column, right below the column name
# Transpose accuracy_quality_pivot
avg_accuracy_quality_pivot_transposed = avg_accuracy_quality_pivot.transpose().reset_index()

# Set name of first column to "technique_name"
avg_accuracy_quality_pivot_transposed = avg_accuracy_quality_pivot_transposed.rename(columns={'index': 'technique_name'})

# Left join hand_labeled_data to avg_accuracy_quality_pivot_transposed on technique_name
avg_accuracy_quality_pivot_transposed = avg_accuracy_quality_pivot_transposed.merge(hand_labeled_data, on='technique_name', how='left')

# Transpose back
avg_accuracy_quality_pivot_with_date = avg_accuracy_quality_pivot_transposed.transpose().reset_index()

# # Set names to last row
# avg_accuracy_quality_pivot_with_date.columns = avg_accuracy_quality_pivot_with_date.iloc[-1]
# # Drop first row, reset index
# avg_accuracy_quality_pivot_with_date = avg_accuracy_quality_pivot_with_date.drop([0]).reset_index().drop(columns=['index'])
# # Drop row where technique_name is "Prompt Engineering Method"
# #avg_accuracy_quality_pivot_with_date = avg_accuracy_quality_pivot_with_date[avg_accuracy_quality_pivot_with_date['technique_name'] != 'Prompt Engineering Method']
# # In first column, set values of "0", "1", "2", "3" to blanks
# #avg_accuracy_quality_pivot_with_date['technique_name'] = avg_accuracy_quality_pivot_with_date['technique_name'].replace({'0': '', '1': '', '2': '', '3': ''})
# # Drop column tecnique_name
# #avg_accuracy_quality_pivot_with_date = avg_accuracy_quality_pivot_with_date.drop(columns=['technique_name'])
# # Drop first column
# avg_accuracy_quality_pivot_with_date = avg_accuracy_quality_pivot_with_date.iloc[:, 1:]
# # Set second column name to "Model"
# avg_accuracy_quality_pivot_with_date = avg_accuracy_quality_pivot_with_date.rename(index={1: 'Model'})
# # Set third column name to "Task"
# avg_accuracy_quality_pivot_with_date = avg_accuracy_quality_pivot_with_date.rename(index={2: 'Task'})
# avg_accuracy_quality_pivot_with_date.rename(columns={avg_accuracy_quality_pivot_with_date.columns[0]: 'Model'}, inplace=True)
# avg_accuracy_quality_pivot_with_date.rename(columns={avg_accuracy_quality_pivot_with_date.columns[1]: 'Task'}, inplace=True)
# Drop bottom row
#avg_accuracy_quality_pivot_with_date = avg_accuracy_quality_pivot_with_date.drop([avg_accuracy_quality_pivot_with_date.index[-1]])
# Move bottom row to top
#avg_accuracy_quality_pivot_with_date = pd.concat([avg_accuracy_quality_pivot_with_date.iloc[[-1]], avg_accuracy_quality_pivot_with_date.iloc[:-1]]).reset_index().drop(columns=['index'])
# Set column names
#new_part = ['Model', 'Task', 'Direct Prompting']
#new_names = new_part.extend(avg_accuracy_quality_pivot_with_date.columns[3:])
#avg_accuracy_quality_pivot_with_date.columns = new_names
# avg_accuracy_quality_pivot_with_date.columns = ['Model', 'Task', 'Zero-Shot Chain-of-Thought', 'Automatic Prompt Engineer', 'Tree-of-Thought',                'Self-Refine',
#           'Least-to-Most Prompting',          'Few-Shot Learning',
#        'Chain-of-Thought Prompting']
print(avg_accuracy_quality_pivot_with_date.columns)

# Output to LaTeX
print(avg_accuracy_quality_pivot_with_date)
avg_accuracy_quality_pivot_with_date.to_latex('../Output/avg_accuracy_quality_pivot_with_date.tex', index=False)


      technique_name ss_publication_date   Prompt Engineering Method
0  ape_zero_shot_cot          2022-11-03   Automatic Prompt Engineer
1      least_to_most          2022-05-21     Least-to-Most Prompting
2         manual_cot          2022-01-28  Chain-of-Thought Prompting
3    manual_few_shot          2020-05-28           Few-Shot Learning
4        self_refine          2023-03-30                 Self-Refine
5    tree_of_thought          2023-05-15             Tree-of-Thought
6      zero_shot_cot          2022-05-24  Zero-Shot Chain-of-Thought
Index(['index', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype='object')
                       index      0      1                 2  \
0             technique_name  model   task  direct_prompting   
1                          0   gpt4     cw        5.71 (7.3)   
2                          1    td3     cw       4.46 (7.79)   
3                          2   gpt4  gsm8k        0.73 (0.2)   
4                          3    td3  gsm8k       0.23 (0.18)   
5